In [179]:
from openai import OpenAI
import os
import pandas as pd
import numpy as np
import ast
import ssl
from scipy.spatial.distance import cosine

ssl._create_default_https_context = ssl._create_unverified_context

%env OPENAI_API_KEY='KEY'

client = OpenAI(
  api_key=os.environ.get("OPENAI_API_KEY"),
)


env: OPENAI_API_KEY='KEY'


In [131]:
df = pd.read_csv('datatran_pi_2024.csv', delimiter=';')

In [132]:
nomes_colunas = df.columns
expressao = ' + ". " + '.join(['"' + coluna + ': " + df.' + coluna + '.astype(str)' for coluna in nomes_colunas])

In [ ]:
df["combined"] = eval(expressao)

In [133]:
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [134]:
df['embedding'] = df.combined.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
df.to_csv('embedded_acidentes.csv', index=False)

In [145]:
df = pd.read_csv('embedded_acidentes.csv')
df['embedding'] = df.embedding.apply(eval).apply(np.array)

In [146]:
def search(df, description, n=3, pprint=True):
    embedding = get_embedding(
        description,
        model="text-embedding-3-small"
    )
    df["similarity"] = df["embedding"].apply(lambda x: cosine(embedding, x))

    results = (
        df.sort_values("similarity", ascending=True)
        .head(n)
        .combined.str.replace("Resumo: ", "")
        .str.replace("; Conteudo:", ": ")
    )
    if pprint:
        for r in results:
            print(r[:1000])
            print()
    return results

In [174]:
pergunta = 'Qual foram as principais causas de acidentes que levaram a obito na BR 343'

In [175]:
results = search(df, pergunta, n=10, pprint=False)

In [176]:
resultados_concatenados = "; ".join(results)

In [177]:
response = client.chat.completions.create(
    messages=[
        {'role': 'system', 'content': pergunta},
        {'role': 'user', 'content': resultados_concatenados},
    ],
    model='gpt-3.5-turbo',
    temperature=0,
)

In [178]:
saida = response.choices[0].message.content
saida.replace("\n", " ")

'As principais causas de acidentes que resultaram em óbito na BR 343 foram:  1. Transitar na contramão, resultando em colisão frontal com vítimas fatais. 2. Reação tardia ou ineficiente do condutor, resultando em colisão lateral no mesmo sentido com vítimas feridas. 3. Chuva, resultando em saída de leito carroçável com vítimas fatais.  Essas foram as causas identificadas nos acidentes que resultaram em óbito na BR 343. É importante ressaltar a importância da prudência, atenção e respeito às leis de trânsito para evitar tais tragédias.'